Due to challenges faced trying to access the blockchain data en masse, i created a local instance of the lbry-explorer and this notebook leverages the mongoDB data source that exists as part of the lbry-explorer.

In [1]:
import pandas as pd
from pymongo import MongoClient
import matplotlib.pyplot as plt
import datetime
%matplotlib inline

In [2]:
import getpass
password = getpass.getpass()

········


In [3]:
def connect_mongo(host, port, username, password, db):
    """ A util for making a connection to mongo """

    if username and password:
        mongo_uri = 'mongodb://%s:%s@%s:%s/%s' % (username, password, host, port, db)
        conn = MongoClient(mongo_uri)
    else:
        conn = MongoClient(host, port)


    return conn[db]


def read_mongo(db, collection, host='localhost', port=27017, username=None, password=None, no_id=True):
    """ Read from Mongo and Store into DataFrame """

    # Connect to MongoDB
    db = connect_mongo(host=host, port=port, username=username, password=password, db=db)
    
    # Make a query to the specific DB and Collection
    cursor = db[collection].find()
    #print list(cursor)

    # Expand the cursor and construct the DataFrame
    df =  pd.DataFrame(list(cursor))

    # Delete the _id
    if no_id:
        del df['_id']

    return df.drop("__v", axis=1)

Collections:
    addresses,
    markets,
    richlists,
    stats,
    txes

In [4]:
db = "explorerdb"

In [5]:
address_df = read_mongo(db, "addresses", host='localhost', port=27017, username="zargham", password=password)

In [6]:
address_df[address_df.a_id.map(lambda s: bool((s[0] != "b") & (s[0] !="r")))].head()

,a_id,balance,received,sent,txs
0,coinbase,0,0,9.244448e+14,"[{u'type': u'vin', u'addresses': u'5dda81f4219..."


In [7]:
labels = {"rEQKyb7nd7UUGyEEn5xRkk1fgXdTCf2ZCg": {"label":"LBRY Team", "type":"info", "url":"https://lbry.io/what#credits"},
      "bWbFyN8CJAghwavfRDuEfmv71swkapBcuy": {"label":"LBRY Hot Wallet", "type":"info", "url":""},
      "rVRMmGLZenVXpT1NwYMGcnwFc642kEgWTV": {"label":"Strategic Partners", "type":"info", "url":"https://lbry.io/what#credits"},
      "bLPbiXBp6Vr3NSnsHzDsLNzoy5o36re9Cz": {"label":"Investor", "type":"info", "url":"https://lbry.io/news/mysterious-100k-lbc-revealed"},
      "rTZF9RvfkLJ6hwCwERwFd7dppRMeddKjpe": {"label":"LBRY Team", "type":"info", "url":"https://lbry.io/what#credits"},
      "bVUrbCK8hcZ5XWti7b9eNxKEBxzc1rr393": {"label":"LBRY Hot Wallet", "type":"info", "url":""},
      "bHW58d37s1hBjj3wPBkn5zpCX3F8ZW3uWf": {"label":"LBRY Team Escrow (premine)", "type":"info", "url":"https://lbry.io/what#credits"},
      "r9DarmxyPjWkF7ocyxMzaNZN3a9gJvNTZJ": {"label":"Strategic Partners", "type":"info", "url":"https://lbry.io/what#credits"},
      "rMT5Sg8SyFP3ax2PRaweRCRZoMeYw4znEi": {"label":"Adoption Programs", "type":"info", "url":"https://lbry.io/what#credits"},
      "bDzuP6Sawe7tadVKR7dmPW1JFYK9ELnjra": {"label":"LBRY Hot Wallet", "type":"info", "url":""},
      "bFgohv4wprdCEtW2m5EVQyVNtxp4NxQMXi": {"label":"LBRY Hot Wallet", "type":"info", "url":""},
      "rRmURzvpHTysU4xUSp9CXeosBTbYfKs7n5": {"label":"Adoption Programs", "type":"info", "url":"https://lbry.io/what#credits"}
    }

In [8]:
def get_label(address):
    if address in labels.keys():
        return labels[address]["label"]
    else:
        return "unknown"

In [9]:
address_df["label"] = address_df.a_id.map(get_label)

In [10]:
address_df[address_df.label != "unknown"]

,a_id,balance,received,sent,txs,label
404,rEQKyb7nd7UUGyEEn5xRkk1fgXdTCf2ZCg,0.000000e+00,1.999000e+16,1.999000e+16,"[{u'type': u'vout', u'addresses': u'32f0ab6b2a...",LBRY Team
406,rMT5Sg8SyFP3ax2PRaweRCRZoMeYw4znEi,0.000000e+00,3.980000e+16,3.980000e+16,"[{u'type': u'vout', u'addresses': u'781c100a80...",Adoption Programs
410,r9DarmxyPjWkF7ocyxMzaNZN3a9gJvNTZJ,0.000000e+00,1.000000e+16,1.000000e+16,"[{u'type': u'vout', u'addresses': u'9600945091...",Strategic Partners
424,bHW58d37s1hBjj3wPBkn5zpCX3F8ZW3uWf,4.000000e+16,0.000000e+00,4.000000e+16,"[{u'type': u'vin', u'addresses': u'9a72043e11b...",LBRY Team Escrow (premine)
4057,bLPbiXBp6Vr3NSnsHzDsLNzoy5o36re9Cz,1.000090e+13,1.000090e+13,0.000000e+00,"[{u'type': u'vout', u'addresses': u'e2f9cb21d4...",Investor
4163,bFgohv4wprdCEtW2m5EVQyVNtxp4NxQMXi,0.000000e+00,7.352650e+13,7.352650e+13,"[{u'type': u'vout', u'addresses': u'7c77646740...",LBRY Hot Wallet
4833,bVUrbCK8hcZ5XWti7b9eNxKEBxzc1rr393,0.000000e+00,4.897770e+13,4.897770e+13,"[{u'type': u'vout', u'addresses': u'ed8fc03abd...",LBRY Hot Wallet
128905,bDzuP6Sawe7tadVKR7dmPW1JFYK9ELnjra,1.450000e+14,2.984199e+14,1.534199e+14,"[{u'type': u'vout', u'addresses': u'554d8fed52...",LBRY Hot Wallet
129116,bWbFyN8CJAghwavfRDuEfmv71swkapBcuy,0.000000e+00,8.419899e+12,8.419899e+12,"[{u'type': u'vout', u'addresses': u'b72ffc5bb9...",LBRY Hot Wallet
236796,rVRMmGLZenVXpT1NwYMGcnwFc642kEgWTV,1.000000e+16,1.000000e+16,1.000000e+06,"[{u'type': u'vout', u'addresses': u'73df395c9f...",Strategic Partners


In [11]:
tx_df = read_mongo(db, "txes", host='localhost', port=27017, username="zargham", password=password)

In [12]:
import pytz
tz = pytz.timezone('America/Montreal')
tx_df["datetime"] = tx_df.timestamp.map(lambda t: datetime.datetime.fromtimestamp(int(t), tz))

In [13]:
#jacks reward function
COIN = 10**8
block_time = 150
blocks_per_ramp = 100
start_ramp_height = 5001
limit = 1083221000 * COIN
subsidy_interval = 1 << 5
max_reductions = 500
peak_reward = 500 * COIN
premine = 400000000 * COIN


def reward(n):
    assert n >= 0
    if n < 5001:
        return COIN
    elif n < 55001:
        r = 1 + ((n - ((n - start_ramp_height) % blocks_per_ramp) - start_ramp_height) / blocks_per_ramp)
        return r * COIN
    else:
        level = int((n - 55001) / subsidy_interval)
        reduction = int((-1 + int((8 * level) + 1) ** 0.5) / 2)
        while level >= ((reduction * reduction + reduction) >> 1):
            if ((reduction * reduction + reduction) >> 1) > level:
                reduction -= 1
            else:
                reduction += 1
        if reduction * COIN >= peak_reward:
            return 0
        return peak_reward - (reduction * COIN)

In [14]:
tx_df["reward"] = tx_df.blockindex.map(reward)/COIN #coin converts Deweys to LBC

In [34]:
tx_df.tail()

,blockhash,blockindex,timestamp,total,txid,vin,vout,datetime,reward,out_address,in_address,out_amount,in_amount,lbry_out,lbry_in
515640,8850fafc3f56faed531ae6ec16e1535336f372f7ef3907...,47937,1474330243,21408225119,f64e36af4821bfeec586fb8fff94cf4fb77ef2f3d6b55d...,"[{u'amount': 21408229639.0, u'addresses': u'bE...","[{u'amount': 20741450344.0, u'addresses': u'ba...",2016-09-19 20:10:43-04:00,430,bZi1WEjGtsdAwuZTnNNTCAZLxhHkiHec4m,coinbase,100000000,100000000,False,False
515641,8850fafc3f56faed531ae6ec16e1535336f372f7ef3907...,47937,1474330243,23788033896,778f276d9036c777ca98062332cc0ac82dadbd14ddacd4...,"[{u'amount': 23788038396.0, u'addresses': u'bb...","[{u'amount': 1015497931, u'addresses': u'bLMFc...",2016-09-19 20:10:43-04:00,430,bZi1WEjGtsdAwuZTnNNTCAZLxhHkiHec4m,coinbase,100000000,100000000,False,False
515642,8850fafc3f56faed531ae6ec16e1535336f372f7ef3907...,47937,1474330243,32744956889,0f56c519968d39a32029da4df368362806b6a5d9aadbbf...,"[{u'amount': 32744961409.0, u'addresses': u'bN...","[{u'amount': 31276699609.0, u'addresses': u'bD...",2016-09-19 20:10:43-04:00,430,bZi1WEjGtsdAwuZTnNNTCAZLxhHkiHec4m,coinbase,100000000,100000000,False,False
515643,8850fafc3f56faed531ae6ec16e1535336f372f7ef3907...,47937,1474330243,1671721785,b6a773e633a3cd48e7330038f00138e3ebcfeab6c42e20...,"[{u'amount': 1671726305, u'addresses': u'baLUG...","[{u'amount': 583909152, u'addresses': u'bKEm8j...",2016-09-19 20:10:43-04:00,430,bZi1WEjGtsdAwuZTnNNTCAZLxhHkiHec4m,coinbase,100000000,100000000,False,False
515644,8850fafc3f56faed531ae6ec16e1535336f372f7ef3907...,47937,1474330243,22740403702,8488089cd1e4e88b975f4314f2fdf5b5dbaacd5543ebc5...,"[{u'amount': 22740408222.0, u'addresses': u'bN...","[{u'amount': 102381852, u'addresses': u'bPXXjt...",2016-09-19 20:10:43-04:00,430,bZi1WEjGtsdAwuZTnNNTCAZLxhHkiHec4m,coinbase,100000000,100000000,False,False


In [49]:
tx_df.vout.map(lambda X: [x['addresses'] for x in X] )

0                      [bZi1WEjGtsdAwuZTnNNTCAZLxhHkiHec4m]
1                      [bZi1WEjGtsdAwuZTnNNTCAZLxhHkiHec4m]
2                      [bZi1WEjGtsdAwuZTnNNTCAZLxhHkiHec4m]
3                      [bZi1WEjGtsdAwuZTnNNTCAZLxhHkiHec4m]
4                      [bZi1WEjGtsdAwuZTnNNTCAZLxhHkiHec4m]
5                      [bZi1WEjGtsdAwuZTnNNTCAZLxhHkiHec4m]
6                      [bZi1WEjGtsdAwuZTnNNTCAZLxhHkiHec4m]
7                      [bZi1WEjGtsdAwuZTnNNTCAZLxhHkiHec4m]
8                      [bZi1WEjGtsdAwuZTnNNTCAZLxhHkiHec4m]
9                      [bZi1WEjGtsdAwuZTnNNTCAZLxhHkiHec4m]
10                     [bZi1WEjGtsdAwuZTnNNTCAZLxhHkiHec4m]
11                     [bZi1WEjGtsdAwuZTnNNTCAZLxhHkiHec4m]
12                     [bZi1WEjGtsdAwuZTnNNTCAZLxhHkiHec4m]
13                     [bZi1WEjGtsdAwuZTnNNTCAZLxhHkiHec4m]
14                     [bZi1WEjGtsdAwuZTnNNTCAZLxhHkiHec4m]
15                     [bZi1WEjGtsdAwuZTnNNTCAZLxhHkiHec4m]
16                     [bZi1WEjGtsdAwuZT

In [38]:
tx_df['out_address'] = 
tx_df['in_address'] = 

TypeError: list indices must be integers, not str

In [15]:
tx_df['out_amount']=
tx_df['in_amount'] = 

In [25]:
lbryKeys = labels.keys()
tx_df['lbry_out'] = tx_df.out_address.map(lambda x: bool(x in lbryKeys))
tx_df['lbry_in'] = tx_df.in_address.map(lambda x: bool(x in lbryKeys)) 

In [26]:
relevent_tx = tx_df[(tx_df.lbry_out==True) | (tx_df.lbry_in==True)]


In [28]:
lbryKeys

['rEQKyb7nd7UUGyEEn5xRkk1fgXdTCf2ZCg',
 'rTZF9RvfkLJ6hwCwERwFd7dppRMeddKjpe',
 'bLPbiXBp6Vr3NSnsHzDsLNzoy5o36re9Cz',
 'r9DarmxyPjWkF7ocyxMzaNZN3a9gJvNTZJ',
 'bWbFyN8CJAghwavfRDuEfmv71swkapBcuy',
 'rRmURzvpHTysU4xUSp9CXeosBTbYfKs7n5',
 'rVRMmGLZenVXpT1NwYMGcnwFc642kEgWTV',
 'bVUrbCK8hcZ5XWti7b9eNxKEBxzc1rr393',
 'rMT5Sg8SyFP3ax2PRaweRCRZoMeYw4znEi',
 'bDzuP6Sawe7tadVKR7dmPW1JFYK9ELnjra',
 'bFgohv4wprdCEtW2m5EVQyVNtxp4NxQMXi',
 'bHW58d37s1hBjj3wPBkn5zpCX3F8ZW3uWf']

In [27]:
relevent_tx

,blockhash,blockindex,timestamp,total,txid,vin,vout,datetime,reward,out_address,in_address,out_amount,in_amount,lbry_out,lbry_in


In [ ]:
tx_df.plot(x="datetime", y= "reward")
plt.ylabel("LBC")

In [ ]:
relevent_tx = pd.DataFrame([row[0] for row in address_df[address_df.label != "unknown"].txs])

In [ ]:
relevent_tx.addresses.values
#these are transaction IDs

We are going to use transaction ids, the blockchain database and the addresses table to create a history of the LBRY addresses named in "labels" and used for operating the business.

In [ ]:
tx_df.blockindex.max()